In [1]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 600
from tqdm import tqdm
import gc

from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.decomposition import PCA

from keras.layers.normalization import BatchNormalization

from keras.models import Sequential, Model

from keras.layers import Input, Embedding, Dense, Activation, Dropout, Flatten

from keras import regularizers 

import keras

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GroupKFold

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Using TensorFlow backend.


In [2]:
train_all = pd.read_csv("./train/12182_feats.csv")
train_all.head()

,group_id,cate3_cd,sku_id,dc_id,date,OrderNonOutlierVolume,dt_onstock_min,reserveState,salesForecast,vendibility1,skuDecomposedTrend,skuDecomposedSeasonal,level3DecomposedTrend,level3DecomposedSeasonal,Curve,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,AvgBundleDiscount,AvgDirectDiscount,AvgDiscount,AvgFreegiftDiscount,AvgSyntheticDiscountA,MaxBundleDiscount,MaxDirectDiscount,MaxDiscount,MaxFreegiftDiscount,MaxSyntheticDiscountA,MinBundleDiscount,MinDirectDiscount,MinDiscount,MinFreegiftDiscount,MinSyntheticDiscountA,PromotionCount,SyntheticGrossPrice,bundlecount,bundleflag,daynumberinpromotion,directdiscountflag,durationinhours,freegiftflag,jdprice,numberhierarchylevel1,numberhierarchylevel2,numberhierarchylevel3,numberproducts,promotionkey,promotionsubtype,promotiontype,stockprice,strongmark,suitflag,syntheticdiscounta_vb,syntheticgrossprice_vb,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,SyntheticPromotionSubType,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,suit_ind,freegift_ind,cnt_period,dd_discount_daily_max,dd_discount_sgp_daily_max,bd_discount_daily_max,suit_discount_new_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_wgt,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,dt,biz,stage
0,12182.0001,12182,1548168,5,2018-08-04,14.0,2017-03-18,0.0,14.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,0.086,0.219,0.179,0.0,-0.346,0.10,0.331,0.331,0.0,-0.097,0.057,0.106,0.057,0.0,-0.546,10.0,29.9,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,33.00,0.0,NaN,NaN,NaN,14.161,12.683,12.067,11.929,9.000,10.857,12.6,17.000,23.0,20.0,8.5,8.0,13.241,12.859,11.989,11.095,11.281,13.166,17.768,NaN,6.357,5.286,6.0,5.667,4.0,5.0,5.0,5.0,7.056,6.276,5.641,5.073,23.0,6.0,11.0,23.0,8.0,32.800,29.520,0.0,7.0,3.0,0.0,0.0,0.0,2.0,0.331,0.219,0.398,0.0,32.8,29.520,49.0,29.9,0.331,-0.097,0.398,0.013,72.714,88.571,99.2,124.667,155.5,135.0,62.0,66.0,85.341,89.201,101.420,126.431,2019-03-18,rdc_9192,train
1,12182.0001,12182,1548168,5,2018-06-10,6.0,2017-03-18,0.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,0.125,0.233,0.197,0.0,-0.230,0.15,0.331,0.331,0.0,-0.025,0.100,0.106,0.100,0.0,-0.378,6.0,32.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,33.00,0.0,NaN,NaN,NaN,14.286,14.922,16.767,13.607,12.286,10.857,11.6,13.000,13.0,9.0,12.5,12.0,14.257,14.671,14.680,13.432,12.425,11.849,11.553,NaN,13.571,10.857,11.6,13.000,13.0,9.0,13.5,12.0,12.960,12.294,11.861,11.556,4.0,97.0,12.0,27.0,3.0,32.800,27.880,0.0,7.0,6.0,0.0,0.0,0.0,3.0,0.331,-0.025,0.431,0.0,32.8,27.880,49.0,29.9,0.331,-0.097,0.431,0.068,93.429,82.286,74.4,65.667,65.0,66.0,85.0,86.0,103.126,90.735,79.780,69.042,2019-03-18,rdc_9192,train
2,12182.0001,12182,1548168,5,2017-07-12,13.0,2017-03-18,0.0,13.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,0.100,0.199,0.174,0.0,-0.038,0.10,0.265,0.265,0.0,0.077,0.100,0.143,0.100,0.0,-0.131,4.0,39.0,1.0,0.0,5.

In [3]:
selected_train_all = train_all[['sku_id', 'dc_id', 'OrderNonOutlierVolume', 'Holiday', 'Ind_1111_pre', 'Ind_1111', 'Ind_1111_post', 'Ind_618_pre', 'Ind_618', 'Ind_618_post', 'Ind_1212', 'Month', 'DayOfWeek', 'PromotionCount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA', 'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount', 'AvgDirectDiscount', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'numberproducts', 'strongmark', 'rolling360Mean', 'rolling180Mean', 'rolling90Mean', 'rolling28Mean', 'rolling14Mean', 'rolling7Mean', 'rolling5Mean', 'rolling3Mean', 'rolling2Mean', 'rolling1Mean', 'rolling14Median', 'rolling7Median', 'rolling360DecayMean', 'rolling180DecayMean', 'rolling90DecayMean', 'rolling28DecayMean', 'rolling14DecayMean', 'rolling7DecayMean', 'rolling3DecayMean', 'national_rolling14Mean', 'national_rolling7Mean', 'national_rolling5Mean', 'national_rolling3Mean', 'national_rolling2Mean', 'national_rolling1Mean', 'national_rolling14Median', 'national_rolling7Median', 'national_rolling28DecayMean', 'national_rolling14DecayMean', 'national_rolling7DecayMean', 'national_rolling3DecayMean', 'similar_rolling14Mean', 'similar_rolling7Mean', 'similar_rolling5Mean', 'similar_rolling3Mean', 'similar_rolling2Mean', 'similar_rolling1Mean', 'similar_rolling14Median', 'similar_rolling7Median', 'similar_rolling28DecayMean', 'similar_rolling14DecayMean', 'similar_rolling7DecayMean', 'similar_rolling3DecayMean', 'volume_rolling365Shift', 'volume_rolling180Shift', 'volume_rolling90Shift', 'volume_rolling30Shift', 'volume_rolling7Shift',  'dd_price_weighted', 'bd_price_weighted', 'flashsale_ind', 'dd_ind', 'bundle_ind', 'bundle_buy199get100_ind',   'cnt_period', 'dd_discount_daily_max', 'bd_discount_daily_max', 'dd_price_daily_min', 'bd_price_daily_min', 'jdprice_daily_min', 'sgp_daily_min', 'dd_discount_sgp_wgt', 'bd_discount_wgt', 'bd_discount_sgp_wgt']]

In [4]:
selected_train_all.fillna(0, inplace=True)
selected_train_all.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,sku_id,dc_id,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt
0,1548168,5,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,10.0,0.331,0.057,0.179,-0.097,-0.546,-0.346,0.10,0.057,0.086,0.331,0.106,0.219,0.000,0.0,0.0,1.0,0.0,14.161,12.683,12.067,11.929,9.000,10.857,12.6,17.000,23.0,20.0,8.5,8.0,13.241,12.859,11.989,11.095,11.281,13.166,17.768,72.714,88.571,99.2,124.667,155.5,135.0,62.0,66.0,85.341,89.201,101.420,126.431,6.357,5.286,6.0,5.667,4.0,5.0,5.0,5.0,7.056,6.276,5.641,5.073,23.0,6.0,11.0,23.0,8.0,32.800,29.520,0.0,7.0,3.0,0.0,2.0,0.331,0.398,32.8,29.520,49.0,29.9,-0.097,0.398,0.013
1,1548168,5,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,0.331,0.100,0.197,-0.025,-0.378,-0.230,0.15,0.100,0.125,0.331,0.106,0.233,0.000,0.0,0.0,1.0,0.0,14.286,14.922,16.767,13.607,12.286,10.857,11.6,13.000,13.0,9.0,12.5,12.0,14.257,14.671,14.680,13.432,12.425,11.849,11.553,93.429,82.286,74.4,65.667,65.0,66.0,85.0,86.0,103.126,90.735,79.780,69.042,13.571,10.857,11.6,13.000,13.0,9.0,13.5,12.0,12.960,12.294,11.861,11.556,4.0,97.0,12.0,27.0,3.0,32.800,27.880,0.0,7.0,6.0,0.0,3.0,0.331,0.431,32.8,27.880,49.0,29.9,-0.097,0.431,0.068
2,1548168,5,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,4.0,0.265,0.100,0.174,0.077,-0.131,-0.038,0.10,0.100,0.100,0.265,0.143,0.199,0.077,24.0,5.0,1.0,0.0,0.000,0.000,7.411,9.214,8.286,8.857,9.8,12.333,11.5,8.0,8.0,8.0,8.123,8.029,8.000,8.656,9.117,9.693,10.326,62.214,67.000,67.0,72.333,73.0,61.0,64.5,68.0,63.848,65.126,66.646,68.017,7.357,8.571,10.4,12.333,11.5,8.0,6.5,8.0,7.489,8.064,9.246,10.327,0.0,0.0,8.0,11.0,4.0,36.000,32.400,0.0,3.0,1.0,0.0,1.0,0.265,0.339,36.0,32.400,49.0,39.0,0.077,0.339,0.169
3,1548168,5,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,5.0,7.0,0.331,0.100,0.185,0.176,-0.108,-0.003,0.10,0.100,0.100,0.331,0.106,0.199,0.176,24.0,1.0,1.0,0.0,12.928,10.483,10.533,13.357,14.571,16.857,18.2,13.667,9.0,11.0,13.0,16.0,11.973,11.461,11.512,13.157,14.356,14.926,13.067,99.786,111.286,133.8,118.333,74.0,60.0,65.5,88.0,97.636,101.894,105.934,95.183,0.000,62.286,80.0,122.667,176.0,310.0,0.0,18.0,60.551,72.753,99.913,169.985,23.0,17.0,5.0,6.0,11.0,32.800,29.520,1.0,5.0,2.0,0.0,2.0,0.331,0.398,32.8,29.520,49.0,39.8,0.176,0.398,0.258
4,1548168,5,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,6.0,9.0,0.331,0.100,0.192,0.178,-0.105,0.008,0.10,0.100,0.100,0.331,0.106,0.218,0.178,22.0,2.0,1.0,0.0,12.897,10.894,9.53

In [5]:
len(selected_train_all['sku_id'].unique())

86

In [6]:
selected_train_all['dc_id'].unique()

array([  5,  10,   3, 772,   6,   4,   9, 316])

In [7]:
# selected_train_all.dc_id.apply(lambda x: x in)
dc_selected = selected_train_all[selected_train_all.dc_id.isin([3,4,5,6,9,10,316,772])]

In [8]:
dc_selected['dc_id'].unique()

array([  5,  10,   3, 772,   6,   4,   9, 316])

In [9]:
dc_selected.head()

,sku_id,dc_id,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt
0,1548168,5,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,10.0,0.331,0.057,0.179,-0.097,-0.546,-0.346,0.10,0.057,0.086,0.331,0.106,0.219,0.000,0.0,0.0,1.0,0.0,14.161,12.683,12.067,11.929,9.000,10.857,12.6,17.000,23.0,20.0,8.5,8.0,13.241,12.859,11.989,11.095,11.281,13.166,17.768,72.714,88.571,99.2,124.667,155.5,135.0,62.0,66.0,85.341,89.201,101.420,126.431,6.357,5.286,6.0,5.667,4.0,5.0,5.0,5.0,7.056,6.276,5.641,5.073,23.0,6.0,11.0,23.0,8.0,32.800,29.520,0.0,7.0,3.0,0.0,2.0,0.331,0.398,32.8,29.520,49.0,29.9,-0.097,0.398,0.013
1,1548168,5,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,0.331,0.100,0.197,-0.025,-0.378,-0.230,0.15,0.100,0.125,0.331,0.106,0.233,0.000,0.0,0.0,1.0,0.0,14.286,14.922,16.767,13.607,12.286,10.857,11.6,13.000,13.0,9.0,12.5,12.0,14.257,14.671,14.680,13.432,12.425,11.849,11.553,93.429,82.286,74.4,65.667,65.0,66.0,85.0,86.0,103.126,90.735,79.780,69.042,13.571,10.857,11.6,13.000,13.0,9.0,13.5,12.0,12.960,12.294,11.861,11.556,4.0,97.0,12.0,27.0,3.0,32.800,27.880,0.0,7.0,6.0,0.0,3.0,0.331,0.431,32.8,27.880,49.0,29.9,-0.097,0.431,0.068
2,1548168,5,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,4.0,0.265,0.100,0.174,0.077,-0.131,-0.038,0.10,0.100,0.100,0.265,0.143,0.199,0.077,24.0,5.0,1.0,0.0,0.000,0.000,7.411,9.214,8.286,8.857,9.8,12.333,11.5,8.0,8.0,8.0,8.123,8.029,8.000,8.656,9.117,9.693,10.326,62.214,67.000,67.0,72.333,73.0,61.0,64.5,68.0,63.848,65.126,66.646,68.017,7.357,8.571,10.4,12.333,11.5,8.0,6.5,8.0,7.489,8.064,9.246,10.327,0.0,0.0,8.0,11.0,4.0,36.000,32.400,0.0,3.0,1.0,0.0,1.0,0.265,0.339,36.0,32.400,49.0,39.0,0.077,0.339,0.169
3,1548168,5,29.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,5.0,7.0,0.331,0.100,0.185,0.176,-0.108,-0.003,0.10,0.100,0.100,0.331,0.106,0.199,0.176,24.0,1.0,1.0,0.0,12.928,10.483,10.533,13.357,14.571,16.857,18.2,13.667,9.0,11.0,13.0,16.0,11.973,11.461,11.512,13.157,14.356,14.926,13.067,99.786,111.286,133.8,118.333,74.0,60.0,65.5,88.0,97.636,101.894,105.934,95.183,0.000,62.286,80.0,122.667,176.0,310.0,0.0,18.0,60.551,72.753,99.913,169.985,23.0,17.0,5.0,6.0,11.0,32.800,29.520,1.0,5.0,2.0,0.0,2.0,0.331,0.398,32.8,29.520,49.0,39.8,0.176,0.398,0.258
4,1548168,5,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,6.0,9.0,0.331,0.100,0.192,0.178,-0.105,0.008,0.10,0.100,0.100,0.331,0.106,0.218,0.178,22.0,2.0,1.0,0.0,12.897,10.894,9.53

In [10]:
df = pd.concat([dc_selected,pd.get_dummies(dc_selected['sku_id'], prefix='sku_id')],axis=1)
df.drop(['sku_id'],axis=1, inplace=True)
df

,dc_id,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,sku_id_1026901,sku_id_1026909,sku_id_1398762,sku_id_1398801,sku_id_1548168,sku_id_1714158,sku_id_1981139,sku_id_1989487,sku_id_2014275,sku_id_2135764,sku_id_2318050,sku_id_2349632,sku_id_2381171,sku_id_2478974,sku_id_2634244,sku_id_3010437,sku_id_3069607,sku_id_3292750,sku_id_3387256,sku_id_3924285,sku_id_3924287,sku_id_3978681,sku_id_4440925,sku_id_4496373,sku_id_4544454,sku_id_4544456,sku_id_4730228,sku_id_4730230,sku_id_4817923,sku_id_4817973,sku_id_4818057,sku_id_5085585,sku_id_5085639,sku_id_5085667,sku_id_5105879,sku_id_5105913,sku_id_5105917,sku_id_5130508,sku_id_5192594,sku_id_5231298,sku_id_5255124,sku_id_5303624,sku_id_5479329,sku_id_5481122,sku_id_5481124,sku_id_5673720,sku_id_6046621,sku_id_6082869,sku_id_6097664,sku_id_6097668,sku_id_6114481,sku_id_6114547,sku_id_6114697,sku_id_6167552,sku_id_6167944,sku_id_6402273,sku_id_6419885,sku_id_6419893,sku_id_6494420,sku_id_6775423,sku_id_6851115,sku_id_7008753,sku_id_7101462,sku_id_7345856,sku_id_7397360,sku_id_7559399,sku_id_7559589,sku_id_7559649,sku_id_7618402,sku_id_7618422,sku_id_7728061,sku_id_7741210,sku_id_7741214,sku_id_7924762,sku_id_8080562,sku_id_8139612,sku_id_8244699,sku_id_8322158,sku_id_8331114,sku_id_8427296,sku_id_8456973,sku_id_8639931,sku_id_8669934,sku_id_8881505,sku_id_8960273,sku_id_8960277
0,5,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,10.0,0.331,0.057,0.179,-0.097,-0.546,-0.346,0.100,0.057,0.086,0.331,0.106,0.219,0.000,0.0,0.0,1.0,0.0,14.161,12.683,12.067,11.929,9.000,10.857,12.6,17.000,23.0,20.0,8.5,8.0,13.241,12.859,11.989,11.095,11.281,13.166,17.768,72.714,88.571,99.2,124.667,155.5,135.0,62.0,66.0,85.341,89.201,101.420,126.431,6.357,5.286,6.0,5.667,4.0,5.0,5.0,5.0,7.056,6.276,5.641,5.073,23.0,6.0,11.0,23.0,8.0,32.800,29.520,0.0,7.0,3.0,0.0,2.0,0.331,0.398,32.8,29.520,49.0,29.9,-0.097,0.398,0.013,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,0.331,0.100,0.197,-0.025,-0.378,-0.230,0.150,0.100,0.125,0.331,0.106,0.233,0.000,0.0,0.0,1.0,0.0,14.286,14.922,16.767,13.607,12.286,10.857,11.6,13.000,13.0,9.0,12.5,12.0,14.257,14.671,14.680,13.432,12.425,11.849,11.553,93.429,82.286,74.4,65.667,65.0,66.0,85.0,86.0,1

In [11]:
df = pd.concat([df,pd.get_dummies(df['dc_id'], prefix='dc_id')],axis=1)
df.drop(['dc_id'],axis=1, inplace=True)
df

,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,sku_id_1026901,sku_id_1026909,sku_id_1398762,sku_id_1398801,sku_id_1548168,sku_id_1714158,sku_id_1981139,sku_id_1989487,sku_id_2014275,sku_id_2135764,sku_id_2318050,sku_id_2349632,sku_id_2381171,sku_id_2478974,sku_id_2634244,sku_id_3010437,sku_id_3069607,sku_id_3292750,sku_id_3387256,sku_id_3924285,sku_id_3924287,sku_id_3978681,sku_id_4440925,sku_id_4496373,sku_id_4544454,sku_id_4544456,sku_id_4730228,sku_id_4730230,sku_id_4817923,sku_id_4817973,sku_id_4818057,sku_id_5085585,sku_id_5085639,sku_id_5085667,sku_id_5105879,sku_id_5105913,sku_id_5105917,sku_id_5130508,sku_id_5192594,sku_id_5231298,sku_id_5255124,sku_id_5303624,sku_id_5479329,sku_id_5481122,sku_id_5481124,sku_id_5673720,sku_id_6046621,sku_id_6082869,sku_id_6097664,sku_id_6097668,sku_id_6114481,sku_id_6114547,sku_id_6114697,sku_id_6167552,sku_id_6167944,sku_id_6402273,sku_id_6419885,sku_id_6419893,sku_id_6494420,sku_id_6775423,sku_id_6851115,sku_id_7008753,sku_id_7101462,sku_id_7345856,sku_id_7397360,sku_id_7559399,sku_id_7559589,sku_id_7559649,sku_id_7618402,sku_id_7618422,sku_id_7728061,sku_id_7741210,sku_id_7741214,sku_id_7924762,sku_id_8080562,sku_id_8139612,sku_id_8244699,sku_id_8322158,sku_id_8331114,sku_id_8427296,sku_id_8456973,sku_id_8639931,sku_id_8669934,sku_id_8881505,sku_id_8960273,sku_id_8960277,dc_id_3,dc_id_4,dc_id_5,dc_id_6,dc_id_9,dc_id_10,dc_id_316,dc_id_772
0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,10.0,0.331,0.057,0.179,-0.097,-0.546,-0.346,0.100,0.057,0.086,0.331,0.106,0.219,0.000,0.0,0.0,1.0,0.0,14.161,12.683,12.067,11.929,9.000,10.857,12.6,17.000,23.0,20.0,8.5,8.0,13.241,12.859,11.989,11.095,11.281,13.166,17.768,72.714,88.571,99.2,124.667,155.5,135.0,62.0,66.0,85.341,89.201,101.420,126.431,6.357,5.286,6.0,5.667,4.0,5.0,5.0,5.0,7.056,6.276,5.641,5.073,23.0,6.0,11.0,23.0,8.0,32.800,29.520,0.0,7.0,3.0,0.0,2.0,0.331,0.398,32.8,29.520,49.0,29.9,-0.097,0.398,0.013,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,0.331,0.100,0.197,-0.025,-0.378,-0.230,0.150,0.100,0.125,0.331,0.106,0.233,0.000,0.0,0.0,1.0,0.0,14.286,14.922,16.767,13.607,12.286,10.857,11.6,13.000,13.0,9.0,12.5,12.0,14.257,14.671,14.680,

In [12]:
x_train = df.iloc[:,1:]
y_train = df['OrderNonOutlierVolume']
# x_train.iloc[0]
# y_train[0]

In [13]:
x_train = np.array(x_train.values.tolist())

In [14]:
x_train[0]

array([ 2.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        8.00000e+00,  5.00000e+00,  1.00000e+01,  3.31000e-01,
        5.70000e-02,  1.79000e-01, -9.70000e-02, -5.46000e-01,
       -3.46000e-01,  1.00000e-01,  5.70000e-02,  8.60000e-02,
        3.31000e-01,  1.06000e-01,  2.19000e-01,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  1.00000e+00,  0.00000e+00,
        1.41610e+01,  1.26830e+01,  1.20670e+01,  1.19290e+01,
        9.00000e+00,  1.08570e+01,  1.26000e+01,  1.70000e+01,
        2.30000e+01,  2.00000e+01,  8.50000e+00,  8.00000e+00,
        1.32410e+01,  1.28590e+01,  1.19890e+01,  1.10950e+01,
        1.12810e+01,  1.31660e+01,  1.77680e+01,  7.27140e+01,
        8.85710e+01,  9.92000e+01,  1.24667e+02,  1.55500e+02,
        1.35000e+02,  6.20000e+01,  6.60000e+01,  8.53410e+01,
        8.92010e+01,  1.01420e+02,  1.26431e+02,  6.35700e+00,
        5.28600e+00,  6.00000e+00,  5.66700e+00,  4.000

In [15]:
y_train = np.array(y_train.values.tolist())

In [16]:
y_train[0]

14.0

In [17]:
test_all = pd.read_csv("./test/12182_feats.csv")
test_all.head()

,group_id,cate3_cd,sku_id,dc_id,date,OrderNonOutlierVolume,dt_onstock_min,reserveState,salesForecast,vendibility1,skuDecomposedTrend,skuDecomposedSeasonal,level3DecomposedTrend,level3DecomposedSeasonal,Curve,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,AvgBundleDiscount,AvgDirectDiscount,AvgDiscount,AvgFreegiftDiscount,AvgSyntheticDiscountA,MaxBundleDiscount,MaxDirectDiscount,MaxDiscount,MaxFreegiftDiscount,MaxSyntheticDiscountA,MinBundleDiscount,MinDirectDiscount,MinDiscount,MinFreegiftDiscount,MinSyntheticDiscountA,PromotionCount,SyntheticGrossPrice,bundlecount,bundleflag,daynumberinpromotion,directdiscountflag,durationinhours,freegiftflag,jdprice,numberhierarchylevel1,numberhierarchylevel2,numberhierarchylevel3,numberproducts,promotionkey,promotionsubtype,promotiontype,stockprice,strongmark,suitflag,syntheticdiscounta_vb,syntheticgrossprice_vb,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,SyntheticPromotionSubType,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,suit_ind,freegift_ind,cnt_period,dd_discount_daily_max,dd_discount_sgp_daily_max,bd_discount_daily_max,suit_discount_new_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_wgt,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,dt,biz,stage
0,12182.0001,12182,2318050,3,2019-03-22,15.0,2018-02-12,0.0,15.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.031,0.031,0.0,-0.033,0.0,0.031,0.031,0.0,-0.033,0.0,0.031,0.031,0.0,-0.033,1.0,30.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,27.2,0.0,NaN,NaN,NaN,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,0.0,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.031,-0.033,0.031,0.0,31.0,31.0,32.0,30.0,0.031,-0.033,0.031,-0.033,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,2019-03-18,rdc_9192,predict
1,12182.0001,12182,2318050,3,2019-03-23,15.0,2018-02-12,0.0,15.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.0,0.031,0.031,0.0,-0.033,0.0,0.031,0.031,0.0,-0.033,0.0,0.031,0.031,0.0,-0.033,1.0,30.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,27.2,0.0,NaN,NaN,NaN,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,0.0,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.031,-0.033,0.031,0.0,31.0,31.0,32.0,30.0,0.031,-0.033,0.031,-0.033,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,2019-03-18,rdc_9192,predict
2,12182.0001,12182,2318050,3,2019-03-21,15.0,2018-02-12,0.0,15.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.031,0.031,0.0,-0.033,0.0,0.031,0.031,0.0,-0.033,0.0,0.031,0.031,0.0,-0.033,1.0,30.0,0.0

In [18]:
selected_test_all = test_all[['sku_id', 'dc_id', 'OrderNonOutlierVolume', 'Holiday', 'Ind_1111_pre', 'Ind_1111', 'Ind_1111_post', 'Ind_618_pre', 'Ind_618', 'Ind_618_post', 'Ind_1212', 'Month', 'DayOfWeek', 'PromotionCount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA', 'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount', 'AvgDirectDiscount', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'numberproducts', 'strongmark', 'rolling360Mean', 'rolling180Mean', 'rolling90Mean', 'rolling28Mean', 'rolling14Mean', 'rolling7Mean', 'rolling5Mean', 'rolling3Mean', 'rolling2Mean', 'rolling1Mean', 'rolling14Median', 'rolling7Median', 'rolling360DecayMean', 'rolling180DecayMean', 'rolling90DecayMean', 'rolling28DecayMean', 'rolling14DecayMean', 'rolling7DecayMean', 'rolling3DecayMean', 'national_rolling14Mean', 'national_rolling7Mean', 'national_rolling5Mean', 'national_rolling3Mean', 'national_rolling2Mean', 'national_rolling1Mean', 'national_rolling14Median', 'national_rolling7Median', 'national_rolling28DecayMean', 'national_rolling14DecayMean', 'national_rolling7DecayMean', 'national_rolling3DecayMean', 'similar_rolling14Mean', 'similar_rolling7Mean', 'similar_rolling5Mean', 'similar_rolling3Mean', 'similar_rolling2Mean', 'similar_rolling1Mean', 'similar_rolling14Median', 'similar_rolling7Median', 'similar_rolling28DecayMean', 'similar_rolling14DecayMean', 'similar_rolling7DecayMean', 'similar_rolling3DecayMean', 'volume_rolling365Shift', 'volume_rolling180Shift', 'volume_rolling90Shift', 'volume_rolling30Shift', 'volume_rolling7Shift',  'dd_price_weighted', 'bd_price_weighted', 'flashsale_ind', 'dd_ind', 'bundle_ind', 'bundle_buy199get100_ind',   'cnt_period', 'dd_discount_daily_max', 'bd_discount_daily_max', 'dd_price_daily_min', 'bd_price_daily_min', 'jdprice_daily_min', 'sgp_daily_min', 'dd_discount_sgp_wgt', 'bd_discount_wgt', 'bd_discount_sgp_wgt']]

In [19]:
selected_test_all.fillna(0, inplace=True)
selected_test_all.head()

,sku_id,dc_id,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt
0,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033
1,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033
2,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033
3,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033
4,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63

In [20]:
test_dc_selected = selected_test_all[selected_test_all.dc_id.isin([3,4,5,6,9,10,316,772])]
test_dc_selected

,sku_id,dc_id,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt
0,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.800,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.670,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033
1,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.800,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.670,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033
2,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.800,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.670,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033
3,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.800,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.670,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033
4,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0

In [21]:
df = pd.concat([test_dc_selected,pd.get_dummies(test_dc_selected['sku_id'], prefix='sku_id')],axis=1)
df.drop(['sku_id'],axis=1, inplace=True)
df

,dc_id,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,sku_id_1026901,sku_id_1026909,sku_id_1398762,sku_id_1398801,sku_id_1548168,sku_id_1714158,sku_id_1981139,sku_id_1989487,sku_id_2014275,sku_id_2135764,sku_id_2318050,sku_id_2349632,sku_id_2381171,sku_id_2478974,sku_id_2634244,sku_id_3010437,sku_id_3069607,sku_id_3292750,sku_id_3387256,sku_id_3924285,sku_id_3924287,sku_id_3978681,sku_id_4440925,sku_id_4496373,sku_id_4544454,sku_id_4544456,sku_id_4730228,sku_id_4730230,sku_id_4817923,sku_id_4817973,sku_id_4818057,sku_id_5085585,sku_id_5085639,sku_id_5085667,sku_id_5105879,sku_id_5105913,sku_id_5105917,sku_id_5130508,sku_id_5192594,sku_id_5231298,sku_id_5255124,sku_id_5303624,sku_id_5479329,sku_id_5481122,sku_id_5481124,sku_id_5673720,sku_id_6046621,sku_id_6082869,sku_id_6097664,sku_id_6097668,sku_id_6114481,sku_id_6114547,sku_id_6114697,sku_id_6167552,sku_id_6167944,sku_id_6402273,sku_id_6419885,sku_id_6419893,sku_id_6494420,sku_id_6775423,sku_id_6851115,sku_id_7008753,sku_id_7101462,sku_id_7345856,sku_id_7397360,sku_id_7559399,sku_id_7559589,sku_id_7559649,sku_id_7618402,sku_id_7618422,sku_id_7728061,sku_id_7741210,sku_id_7741214,sku_id_7924762,sku_id_8080562,sku_id_8139612,sku_id_8244699,sku_id_8322158,sku_id_8331114,sku_id_8427296,sku_id_8456973,sku_id_8639931,sku_id_8669934,sku_id_8881505,sku_id_8960273,sku_id_8960277
0,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.800,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.670,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.800,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,30

In [22]:
df = pd.concat([df,pd.get_dummies(df['dc_id'], prefix='dc_id')],axis=1)
df.drop(['dc_id'],axis=1, inplace=True)
df

,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,sku_id_1026901,sku_id_1026909,sku_id_1398762,sku_id_1398801,sku_id_1548168,sku_id_1714158,sku_id_1981139,sku_id_1989487,sku_id_2014275,sku_id_2135764,sku_id_2318050,sku_id_2349632,sku_id_2381171,sku_id_2478974,sku_id_2634244,sku_id_3010437,sku_id_3069607,sku_id_3292750,sku_id_3387256,sku_id_3924285,sku_id_3924287,sku_id_3978681,sku_id_4440925,sku_id_4496373,sku_id_4544454,sku_id_4544456,sku_id_4730228,sku_id_4730230,sku_id_4817923,sku_id_4817973,sku_id_4818057,sku_id_5085585,sku_id_5085639,sku_id_5085667,sku_id_5105879,sku_id_5105913,sku_id_5105917,sku_id_5130508,sku_id_5192594,sku_id_5231298,sku_id_5255124,sku_id_5303624,sku_id_5479329,sku_id_5481122,sku_id_5481124,sku_id_5673720,sku_id_6046621,sku_id_6082869,sku_id_6097664,sku_id_6097668,sku_id_6114481,sku_id_6114547,sku_id_6114697,sku_id_6167552,sku_id_6167944,sku_id_6402273,sku_id_6419885,sku_id_6419893,sku_id_6494420,sku_id_6775423,sku_id_6851115,sku_id_7008753,sku_id_7101462,sku_id_7345856,sku_id_7397360,sku_id_7559399,sku_id_7559589,sku_id_7559649,sku_id_7618402,sku_id_7618422,sku_id_7728061,sku_id_7741210,sku_id_7741214,sku_id_7924762,sku_id_8080562,sku_id_8139612,sku_id_8244699,sku_id_8322158,sku_id_8331114,sku_id_8427296,sku_id_8456973,sku_id_8639931,sku_id_8669934,sku_id_8881505,sku_id_8960273,sku_id_8960277,dc_id_3,dc_id_4,dc_id_5,dc_id_6,dc_id_9,dc_id_10,dc_id_316,dc_id_772
0,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.800,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.670,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.800,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488

In [23]:
x_test = df.iloc[:,1:]
y_test = df['OrderNonOutlierVolume']
# x_test.iloc[0]
# y_train[0]

In [24]:
x_test = np.array(x_test.values.tolist())

In [25]:
x_test[0]

array([ 2.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        3.00000e+00,  4.00000e+00,  1.00000e+00,  3.10000e-02,
        3.10000e-02,  3.10000e-02, -3.30000e-02, -3.30000e-02,
       -3.30000e-02,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        3.10000e-02,  3.10000e-02,  3.10000e-02,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  1.00000e+00,  0.00000e+00,
        5.79060e+01,  6.37110e+01,  5.58000e+01,  7.17860e+01,
        6.87860e+01,  6.47140e+01,  5.78000e+01,  3.53330e+01,
        1.40000e+01,  1.50000e+01,  6.75000e+01,  7.80000e+01,
        6.06640e+01,  6.14940e+01,  6.14880e+01,  6.51520e+01,
        6.22670e+01,  5.18390e+01,  3.16290e+01,  2.67786e+02,
        2.57429e+02,  2.29400e+02,  1.48000e+02,  7.10000e+01,
        5.80000e+01,  2.83500e+02,  3.02000e+02,  2.54841e+02,
        2.64692e+02,  2.61241e+02,  2.59527e+02,  7.06430e+01,
        6.54290e+01,  5.78000e+01,  3.53330e+01,  1.400

In [26]:
y_test = np.array(y_test.values.tolist())

In [27]:
y_test[0]

15.0

In [80]:
import keras.backend as K

def mape_error(y_true, y_pred):
    return K.mean(K.clip(K.abs(y_pred - y_true)/y_true,  0.0, 1.0), axis=-1)

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

def get_model(input_dim, output_dim):
    
    dropout = 0.5
    regularizer = 0.00004
    main_input = Input(shape=(input_dim,), dtype='float32', name='main_input')
#     site_input = Input(shape=(num_sites,), dtype='float32', name='site_input')
#     access_input = Input(shape=(num_accesses,), dtype='float32', name='access_input')
    
    
#     x0 = keras.layers.concatenate([main_input])
    x = Dense(400, activation='relu', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(main_input)
    x = Dropout(dropout)(x)
#     x = keras.layers.concatenate([x0, x])
    x = Dense(400, activation='relu', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x)
    x = BatchNormalization(beta_regularizer=regularizers.l2(regularizer),
                           gamma_regularizer=regularizers.l2(regularizer)
                          )(x)
    x = Dropout(dropout)(x)
    x = Dense(200, activation='relu', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x)
    x = Dropout(dropout)(x)

    x = Dense(100, activation='relu', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x)
    x = Dropout(dropout)(x)
    x = Dense(output_dim, activation='linear', 
              kernel_initializer='lecun_uniform', kernel_regularizer=regularizers.l2(regularizer))(x)

    model =  Model(inputs=[main_input], outputs=[x])
    model.compile(loss=mean_squared_error, optimizer='adam', metrics = [mape_error])
    return model

batch_size=4096

model = get_model(input_dim = 186, output_dim = 1)

print('batch size:', batch_size)

model.fit(x_train, y_train, batch_size=batch_size, epochs=200, verbose=1, callbacks=None, validation_data=(x_test,y_test), shuffle=True)

y_predict = model.predict(x_test, batch_size=batch_size)


batch size: 4096
Train on 70533 samples, validate on 1015 samples
Epoch 1/200
70533/70533 [==============================] - 2s 23us/step - loss: 214.7275 - mape_error: 0.8209 - val_loss: 76.6716 - val_mape_error: 0.7010
Epoch 2/200
70533/70533 [==============================] - 0s 4us/step - loss: 82.8106 - mape_error: 0.6659 - val_loss: 110.1293 - val_mape_error: 0.6792
Epoch 3/200
70533/70533 [==============================] - 0s 4us/step - loss: 62.8858 - mape_error: 0.6476 - val_loss: 81.4312 - val_mape_error: 0.6506
Epoch 4/200
70533/70533 [==============================] - 0s 4us/step - loss: 59.2947 - mape_error: 0.6518 - val_loss: 96.0177 - val_mape_error: 0.7152
Epoch 5/200
70533/70533 [==============================] - 0s 4us/step - loss: 56.9036 - mape_error: 0.6414 - val_loss: 99.9120 - val_mape_error: 0.7147
Epoch 6/200
70533/70533 [==============================] - 0s 4us/step - loss: 55.1864 - mape_error: 0.6402 - val_loss: 118.7659 - val_mape_error: 0.7276
Epoch 7/200


In [81]:
y_predict[0]

array([11.765198], dtype=float32)

In [82]:
test_dc_selected['predict'] = y_predict

In [83]:
test_dc_selected.head()

,sku_id,dc_id,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,predict,unique_id,volume_sum,volume_sum_predict
0,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,11.765198,2318050_3,105.0,64.410248
1,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,11.657765,2318050_3,105.0,64.410248
2,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,11.874323,2318050_3,105.0,64.410248
3,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,11.

In [84]:
test_dc_selected['unique_id'] = test_dc_selected['sku_id'].astype('str')+'_'+test_dc_selected['dc_id'].astype('str')

In [85]:
test_dc_selected['volume_sum'] = test_dc_selected.groupby(['unique_id'])['OrderNonOutlierVolume'].transform('sum')

In [86]:
test_dc_selected['volume_sum_predict'] = test_dc_selected.groupby(['unique_id'])['predict'].transform('sum')

In [87]:
test_dc_selected.head()

,sku_id,dc_id,OrderNonOutlierVolume,Holiday,Ind_1111_pre,Ind_1111,Ind_1111_post,Ind_618_pre,Ind_618,Ind_618_post,Ind_1212,Month,DayOfWeek,PromotionCount,MaxDiscount,MinDiscount,AvgDiscount,MaxSyntheticDiscountA,MinSyntheticDiscountA,AvgSyntheticDiscountA,MaxBundleDiscount,MinBundleDiscount,AvgBundleDiscount,MaxDirectDiscount,MinDirectDiscount,AvgDirectDiscount,syntheticdiscounta_vb,durationinhours,daynumberinpromotion,numberproducts,strongmark,rolling360Mean,rolling180Mean,rolling90Mean,rolling28Mean,rolling14Mean,rolling7Mean,rolling5Mean,rolling3Mean,rolling2Mean,rolling1Mean,rolling14Median,rolling7Median,rolling360DecayMean,rolling180DecayMean,rolling90DecayMean,rolling28DecayMean,rolling14DecayMean,rolling7DecayMean,rolling3DecayMean,national_rolling14Mean,national_rolling7Mean,national_rolling5Mean,national_rolling3Mean,national_rolling2Mean,national_rolling1Mean,national_rolling14Median,national_rolling7Median,national_rolling28DecayMean,national_rolling14DecayMean,national_rolling7DecayMean,national_rolling3DecayMean,similar_rolling14Mean,similar_rolling7Mean,similar_rolling5Mean,similar_rolling3Mean,similar_rolling2Mean,similar_rolling1Mean,similar_rolling14Median,similar_rolling7Median,similar_rolling28DecayMean,similar_rolling14DecayMean,similar_rolling7DecayMean,similar_rolling3DecayMean,volume_rolling365Shift,volume_rolling180Shift,volume_rolling90Shift,volume_rolling30Shift,volume_rolling7Shift,dd_price_weighted,bd_price_weighted,flashsale_ind,dd_ind,bundle_ind,bundle_buy199get100_ind,cnt_period,dd_discount_daily_max,bd_discount_daily_max,dd_price_daily_min,bd_price_daily_min,jdprice_daily_min,sgp_daily_min,dd_discount_sgp_wgt,bd_discount_wgt,bd_discount_sgp_wgt,predict,unique_id,volume_sum,volume_sum_predict
0,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,11.765198,2318050_3,105.0,83.148087
1,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,11.657765,2318050_3,105.0,83.148087
2,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,11.874323,2318050_3,105.0,83.148087
3,2318050,3,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,0.031,0.031,0.031,-0.033,-0.033,-0.033,0.0,0.0,0.0,0.031,0.031,0.031,0.0,0.0,0.0,1.0,0.0,57.906,63.711,55.8,71.786,68.786,64.714,57.8,35.333,14.0,15.0,67.5,78.0,60.664,61.494,61.488,65.152,62.267,51.839,31.629,267.786,257.429,229.4,148.0,71.0,58.0,283.5,302.0,254.841,264.692,261.241,259.527,70.643,65.429,57.8,35.333,14.0,15.0,67.5,78.0,66.164,63.153,52.242,31.67,51.0,54.0,58.0,50.0,112.0,31.0,31.0,0.0,1.0,0.0,0.0,1.0,0.031,0.031,31.0,31.0,32.0,30.0,-0.033,0.031,-0.033,11.

In [88]:
a =test_dc_selected.groupby(['unique_id']).apply(lambda x:np.abs(x['volume_sum']-x['volume_sum_predict'])/(x['volume_sum']))

In [89]:
result = test_dc_selected.iloc[:, -3:]
result_final = result.drop_duplicates()

mape =result_final.groupby(['unique_id']).apply(lambda x:np.abs(x['volume_sum']-x['volume_sum_predict'])/(x['volume_sum']))
remape = mape.replace(np.inf, 0)

In [92]:
result_final.to_csv('12182_dnn_predict.csv')

In [90]:
remape

unique_id        
1026901_9    840     0.417931
1026909_4    119     0.411304
1026909_9    574     0.158316
1398762_5    266     0.019251
1398762_9    707     2.506452
                       ...   
8881505_6    819     0.322904
8960273_316  826     0.377338
8960273_6    833     0.339448
8960273_9    112     0.268764
8960277_4    1008    0.435867
Length: 145, dtype: float64

In [91]:
remape[remape!=0].mean() # 500 epoch

0.4779572633434041

In [96]:
remape[remape!=0].mean() # 200 epoch

0.5292383823310456

In [108]:
remape[remape!=0].mean() # 100 epoch

0.525576963933256